# OSNAP mooring array

In [ ]:
import sys
sys.path.append("..")
import numpy as np
import xarray as xr
import cmocean
import cartopy
import cartopy.crs as ccrs
import matplotlib as mpl
import matplotlib.pyplot as plt

In [ ]:
%matplotlib inline
%config InlineBackend.print_figure_kwargs={'bbox_inches':None}
%load_ext autoreload
%autoreload 2
%aimport - numpy - matplotlib.pyplot

In [ ]:
from tqdm import tqdm_notebook
from paths import file_ex_ocn_ctrl, path_results
from regions import OSNAP
from xr_DataArrays import xr_DZ

In [ ]:
ds = xr.open_dataset(file_ex_ocn_ctrl, decode_times=False)

## mapping the domain

In [ ]:
NLAT = ds.TLAT.copy()
X, Y = np.meshgrid(np.arange(3600), np.arange(2400))
NLAT.values = Y

In [ ]:
DZT = xr_DZ('ocn')

In [ ]:
fig = plt.figure(figsize=(12,4))
ax = plt.axes((.05,.15,.9,.83),projection=ccrs.PlateCarree())
ax.set_extent([-65, -5, 50, 65], crs=ccrs.PlateCarree())

cax, kw = mpl.colorbar.make_axes(ax,location='bottom',pad=0.1,shrink=0.8)

ax.contour(ds.TLONG[:,200:1050], ds.TLAT[:,200:1050], NLAT[:,200:1050].where(ds.REGION_MASK[:,200:1050]>0),
           np.linspace(1700,1960,11), linewidths=1, colors='r', vmin=1700, vmax=1900,
           transform=ccrs.PlateCarree() )

im = ax.pcolormesh(ds.TLONG, ds.TLAT, DZT.sum(dim='z_t').where(ds.REGION_MASK>0),
                   cmap=cmocean.cm.deep, vmin=0, vmax=4500,
                   transform=ccrs.PlateCarree() )

for mooring in OSNAP:
    ax.scatter(OSNAP[mooring][0], OSNAP[mooring][1],
               color='r', transform=ccrs.PlateCarree())

ax.coastlines(resolution='50m')
ax.gridlines(draw_labels=True)

cbar = fig.colorbar(im, cax=cax, extend='max', **kw)
cbar.ax.tick_params(labelsize=14)
label = cbar.set_label('depth [m]', size=16)

plt.savefig(f'{path_results}/OSNAP/OSNAP_domain')

## selecting data from files

In [ ]:
OSNAP

In [ ]:
# def find_nlon_nlat(lon, lat):
#     A = np.where((ds.TLAT.values>lat-.05) & (ds.TLAT.values<lat+.05) & (ds.TLONG.values>lon-.05) & (ds.TLONG.values<lon+.05))
#     print(A)
#     nlon, nlat = A[0][0], A[1][0]
#     return nlon, nlat


In [ ]:
from grid import find_indices_near_coordinate

In [ ]:
for key in tqdm_notebook(OSNAP):
    print(find_indices_near_coordinate(ds, lat=OSNAP[key][1], lon=OSNAP[key][0]+360.))

In [ ]:
from paths import CESM_filename

In [ ]:
CESM_filename(domain='ocn', run='ctrl', m=1, y=1)

In [ ]:
ds = xr.open_dataset('/projects/0/prace_imau/prace_2013081679/cesm1_0_4/spinup_pd_maxcores_f05_t12/OUTPUT/ocn/hist/monthly/spinup_pd_maxcores_f05_t12.pop.h.0001-01.nc', decode_times=False)

In [ ]:
ds.ANGLE.plot()

In [ ]:
plt.figure(figsize=(20,20))
ds.TLAT.where((ds.TLAT.values>58.95) & (ds.TLAT.values<59.05) & (ds.TLONG.values>360-33.05) & (ds.TLONG.values<360-32.95)).plot()